In [36]:
import re, html, json
import requests
from urllib.error import HTTPError
from openai import OpenAI
from IPython.display import Markdown, display, update_display
from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound, TranscriptsDisabled, VideoUnavailable

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"
api_key='ollama'

def yt_title_desc_transcript(url: str, lang="en"):
    """
    Returns {"title": str|None, "description": str|None, "transcript": str|None}.
    - Title via oEmbed (no API key).
    - Description scraped from the watch page (shortDescription).
    - Transcript via youtube-transcript-api, gracefully handling 400/disabled.
    """
    # --- extract 11-char video id ---
    m = re.search(r"(?:v=|/)([0-9A-Za-z_-]{11})|^([0-9A-Za-z_-]{11})$", url)
    vid = (m.group(1) or m.group(2)) if m else None
    if not vid:
        return {"title": None, "description": None, "transcript": None}

    # --- title via oEmbed (very robust) ---
    title = None
    try:
        r = requests.get("https://www.youtube.com/oembed",
                         params={"url": f"https://www.youtube.com/watch?v={vid}", "format": "json"},
                         timeout=10)
        if r.ok:
            title = r.json().get("title")
    except Exception:
        pass

    # --- description from watch page (shortDescription in initial JSON) ---
    description = None
    try:
        page = requests.get(f"https://www.youtube.com/watch?v={vid}", timeout=10).text
        # Look for ytInitialPlayerResponse JSON
        jmatch = re.search(r"ytInitialPlayerResponse\s*=\s*({.*?});", page, re.DOTALL)
        if jmatch:
            data = json.loads(jmatch.group(1))
            desc = data.get("videoDetails", {}).get("shortDescription")
            if desc:
                description = html.unescape(desc)
    except Exception:
        pass

    # --- transcript (handle 400 cleanly) ---
    transcript_text = None
    try:
        items = YouTubeTranscriptApi.get_transcript(vid, languages=[lang])
        transcript_text = " ".join(ch["text"].strip() for ch in items if ch.get("text"))
    except (NoTranscriptFound, TranscriptsDisabled, VideoUnavailable, HTTPError):
        # HTTPError covers the "HTTP Error 400: Bad Request" case
        transcript_text = None
    except Exception:
        transcript_text = None

    return {"title": title, "description": description, "transcript": transcript_text}


In [32]:

system_prompt = '''
You are an assistant that generates detailed yet concise summaries of YouTube videos.
When the user provides a title and description of a YouTube video, your task is to write a coherent, engaging, and informative summary of around 500 words.
The summary should:

Capture the main themes and key points the video likely covers.

Expand on the description logically, providing context and flow.

Stay neutral, factual, and clear (no personal opinions).

Be self-contained so it makes sense without needing to watch the video.
'''

In [33]:
def user_prompt(title, description):
    prompt = '''Provide me the YouTube video title and description.\n
    I will generate a clear, engaging, and concise summary of the video content in around 500 words,\n
    highlighting the main ideas, key points, and important details.\n'''
    prompt += f'here is the title : {title} \n Description : {description} '
    return prompt

In [34]:
def stream_youtube(yt_url):
    ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    video_metadata = yt_title_desc_transcript(yt_url)
    stream = ollama.chat.completions.create(
        model=MODEL,
        messages = [
            {"role":"system", "content": system_prompt},
            {"role":"user", "content": user_prompt(video_metadata['title'], video_metadata['description'])}
        ],
        stream=True
        
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [37]:
stream_youtube('https://youtu.be/99NUJ1cLbBI?list=RDdJ6_aU6auZc')

Here is a summary of the YouTube video:

**Monta Re: A Baul-Inspired Tribute to the Mystic Guru Shankaracharya**

The music video for "Monta Re" by Amit Trivedi, featuring Swanand Kirkire and Amitabh Bhattacharya, is a soulful tribute to the mystic guru Shankaracharya. Set in the Bengali folk music tradition, this song brings to life the ancient tales of Shankaracharya's spiritual journey.

With elegant lyrics penned by Amitabh Bhattacharya, "Monta Re" transports listeners to the banks of the Ganges River, where Shankaracharya wandered in search of wisdom and inner peace. The song's haunting melodies and emotive vocals evoke a sense of longing and introspection, perfectly capturing the mystic guru's spiritual essence.

The music video beautifully illustrates the baul-inspired style, with intricate traditional dance movements performed by a group of energetic dancers. The choreography seamlessly blends elements of Bengal's folk heritage with modern sensibilities, making the song an engaging watch for audience members interested in Indian classical music.

**Music and Lyric Credit:**
Amit Trivedi handles the music composition, ensuring that the melody complements the song's themes without overpowering them. Amitabh Bhattacharya takes credit for the lyrics, which tell stunning stories of Shankaracharya's spiritual adventures. The song features Swanand Kirkire and Amitabh Bhattacharya as vocalists, further enriching its emotional impact.

**Relevance to Bengali Culture:**
"Monta Re" is a heartwarming tribute to Bengal's rich cultural heritage. Inspired by the baul traditions of the region, this song honors Shankaracharya's life and spiritual journey without diminishing his significance in modern times. By showcasing these folk roots, "Monta Re" provides fans with an enriching sensory experience.

You can listen to "Monta Re" along with other T-Series music videos released by Amit Trivedi at the links provided below:

- Watch "Ankahee"
- Check out "Sawaar Loon"
- Explore "Zinda Hoon"

Follow the official T-SERIES YouTube channel for an ever-growing variety of original music tracks!

By embracing the richness of Bengali folk traditions, "Monta Re" embodies a musical reflection of Shankaracharya's extraordinary journey as both spiritual guide and symbol of timeless wisdom.